# Cas non linéaire
---
**Aymen.B**

In [1]:
import sys, pathlib
sys.path.append(str(pathlib.Path.cwd().parent.parent.parent)+"\AYMEN\EXPLORATION\CAS NON LINEAIRE\OUTILS")

## Problème à résoudre

À partir des données $(X, y)$, on veut estimer une fonction $\mu_\theta : \mathbb{R}^p \to \mathbb{R}$ paramétrée par un réseau de neurones artificiel (ANN) à une couche cachée, tel que :

$$y=\mu_\theta(X)+e$$

où :

- $X \in \mathbb{R}^{n \times p}$ est la matrice des variables explicatives,

- $y \in \mathbb{R}^n$ est le vecteur des observations,

- $e \in \mathbb{R}^n$ est un bruit gaussien i.i.d. : $e \sim \mathcal{N}(0, \sigma^2 I_n)$,

- $\mu_\theta(x) = W_2 \cdot \sigma(W_1 x + b_1) + c$ est un MLP avec :
    - $W_1 \in \mathbb{R}^{p_2 \times p}$ les poids de la première couche,
    - $b_1 \in \mathbb{R}^{p_2}$ le biais de la couche cachée,
    - $W_2 \in \mathbb{R}^{1 \times p_2}$ les poids de la couche de sortie,
    - $c \in \mathbb{R}$ l’ordonnée à l’origine,
    - $\sigma$ une fonction d’activation non linéaire dérivable telle que $\sigma(0) = 0$ et $\sigma'(0) > 0$ (e.g. ELU).

## Génération des données

In [4]:
from non_linear_tools import generate_data

n, p, s = 100, 20, 6
sigma = 0.5
seed = 42

y, X, beta = generate_data(n, p, s, sigma, seed=seed)

print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")
print(f"beta shape: {beta.shape}")
print(f"\nbeta: {beta}")

X shape: (100, 20)
y shape: (100,)
beta shape: (20,)

beta: [ 10. -10.  10. -10.  10. -10.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.]


## Evaluation des modèles

on reprends les mêmes test que pour le cas linéaire simplement voici le support estimé :
$$\widehat{S}=\{j\in\{1,\ldots,p\}\mid\|W_{1,:,j}\|_2>0\}$$
et le support exact toujours :
$$S^*=\{j\in\{1,\ldots,p\}\mid\beta_j\neq0\}$$

### Test pesr pour l'exemple

In [ ]:
import numpy as np
from non_linear_tools import pesr  # à adapter selon l'endroit où tu ranges la fonction

# Support réel (par exemple sur 7 variables)
beta = np.array([10, 10, 0, 0, 10, 0, 0])

# Cas 1 : Support parfaitement retrouvé
W1_hat_1 = np.zeros((5, len(beta)))  # 5 neurones cachés
W1_hat_1[:, 0] = 1.0  # active x0
W1_hat_1[:, 1] = 0.5  # active x1
W1_hat_1[:, 4] = -0.8  # active x4

# Cas 2 : Faux positif sur x2 + oubli de x1
W1_hat_2 = np.zeros((5, len(beta)))
W1_hat_2[:, 0] = 1.0  # x0 OK
W1_hat_2[:, 2] = 0.7  # x2 FP
W1_hat_2[:, 4] = -0.8  # x4 OK

# Affichage clair
print("Test 1 - Perfect support match")
print(f"beta        : {beta}")
print(f"Support estimé : {[j for j in range(len(beta)) if np.linalg.norm(W1_hat_1[:, j]) > 0]}")
print(f"PESR         : {pesr(beta, W1_hat_1)}\n")

print("Test 2 - One FP (x2), one FN (x1)")
print(f"beta        : {beta}")
print(f"Support estimé : {[j for j in range(len(beta)) if np.linalg.norm(W1_hat_2[:, j]) > 0]}")
print(f"PESR         : {pesr(beta, W1_hat_2)}")

Test 1 - Perfect support match
beta        : [10 10  0  0 10  0  0]
W1_hat[:,j] : active if ||W1[:,j]|| > 0
Support estimé : [0, 1, 4]


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

## Affichage des résulats

In [ ]:
from non_linear_tools import plot_scores

# Create sample data
x_range = list(range(1, 11))
scores = {
    'pesr': [0.8, 0.85, 0.9, 0.87, 0.83, 0.79, 0.75, 0.72, 0.68, 0.65],
    'f1': [0.75, 0.82, 0.88, 0.85, 0.80, 0.76, 0.73, 0.70, 0.65, 0.62],
    'tprdeez': [0.85, 0.88, 0.92, 0.89, 0.86, 0.82, 0.78, 0.75, 0.71, 0.68]
}

# Test the function
plot_scores(scores, x_range, "Test Scores")

In [ ]:
from non_linear_tools import plot_simulations

# Create sample data
x_range = list(range(1, 11))
scores_1 = {
    'pesr': [0.8, 0.85, 0.9, 0.87, 0.83, 0.79, 0.75, 0.72, 0.68, 0.65],
    'f1': [0.75, 0.82, 0.88, 0.85, 0.80, 0.76, 0.73, 0.70, 0.65, 0.62],
    'tprdeez': [0.85, 0.88, 0.92, 0.89, 0.86, 0.82, 0.78, 0.75, 0.71, 0.68]
}
scores_2 = {
    'pesr': [0.5, 0.6, 0.7, 0.65, 0.6, 0.58, 0.55, 0.53, 0.5, 0.48],
    'f1': [0.45, 0.55, 0.65, 0.63, 0.6, 0.58, 0.56, 0.54, 0.52, 0.5],
    'tprdeez': [0.6, 0.65, 0.7, 0.68, 0.66, 0.64, 0.62, 0.6, 0.58, 0.56],
}

# Test the function
simulations = {
    'score n°1': scores_1,
    'score n°2': scores_2
}

plot_simulations(simulations, x_range, "Test Simulations")